In [ ]:
!pip install google-generativeai python-pptx pillow requests python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 13.3 MB/s eta 0:00:00


In [ ]:
import os

In [ ]:
import google.generativeai as genai
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import requests
from PIL import Image
import io
from dotenv import load_dotenv
import json

load_dotenv()

False

In [44]:
class PPTGenerator:
  def __init__(self, api_key = None):
     self.api_key = api_key
     if not self.api_key:
      raise ValueError("Gemini API key required")

        # Configure Gemini
     genai.configure(api_key=self.api_key)
     self.model = genai.GenerativeModel('gemini-2.5-pro')
     self.model_vision = genai.GenerativeModel('gemini-2.5-pro')

     self.presentation = Presentation()

  def generate_content_outline(self, topic, num_slides=5):
        """Generate content outline using Gemini"""
        prompt = f"""
        Create a detailed outline for a PowerPoint presentation on "{topic}" with {num_slides} slides.
        Return the response as a JSON array with the following structure:
        [
            {{
                "title": "Slide Title",
                "content": "Main content points as bullet points",
                "slide_type": "title|content|image|conclusion"
            }}
        ]

        Make sure the content is engaging, informative, and well-structured.
        The response must be a valid JSON array.
        """

        try:
            response = self.model.generate_content(prompt)
            content = response.text.strip()


            if "```json" in content:
                content = content.split("```json")[1].split("```")[0].strip()
            elif "```" in content:
                content = content.split("```")[1].strip()


            if not content.startswith('[') or not content.endswith(']'):
                # If it's not a valid JSON string, try to parse it as JSON directly
                try:
                    return json.loads(content)
                except json.JSONDecodeError as e:
                    print(e)
                    return None

            try:
                return json.loads(content)
            except json.JSONDecodeError as e:
                print(e)
                return None

        except Exception as e:
            print(e)
            return None

  def generate_image_description(self, slide_content):

        prompt = f"""
        Based on this slide content, suggest a relevant image description that would enhance the presentation:

        {slide_content}

        Return only a brief, descriptive phrase suitable for image search (max 5 words).
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text.strip()
            return content
        except Exception as e:
            print(e)
            return "professional presentation"

  def download_image(self, query, save_path="temp_image.jpg"):
        try:
            url = "https://api.pexels.com/v1/search"
            headers = {
                'Authorization': '' #enter pexel api
            }
            params = {
                'query': query,
                'per_page': 1,
                'orientation': 'landscape'
            }
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()

            data = response.json()
            if not data.get('photos'):
                raise ValueError(f"No images found for query: {query}")

            image_url = data['photos'][0]['src']['original']


            img_response = requests.get(image_url)
            img_response.raise_for_status()


            with open(save_path, 'wb') as f:
                f.write(img_response.content)

            return save_path

        except Exception as e:
            print(e)
            return None

  def create_title_slide(self, title, subtitle=""):

        slide_layout = self.presentation.slide_layouts[0]
        slide = self.presentation.slides.add_slide(slide_layout)


        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(30)
        title_shape.text_frame.paragraphs[0].font.bold = True
        title_shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
        title_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER


        if subtitle:
            subtitle_shape = slide.placeholders[1]
            subtitle_shape.text = subtitle
            subtitle_shape.text_frame.paragraphs[0].font.size = Pt(20)
            subtitle_shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
            subtitle_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

  def create_content_slide(self, title, content, include_image=False):
        slide_layout = self.presentation.slide_layouts[1]
        slide = self.presentation.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(30)
        title_shape.text_frame.paragraphs[0].font.bold = True


        content_shape = slide.placeholders[1]
        content_shape.text = content


        text_frame = content_shape.text_frame
        for paragraph in text_frame.paragraphs:
            paragraph.font.size = Pt(20)
            paragraph.font.color.rgb = RGBColor(0, 0, 0)


        if include_image:
            try:
                image_desc = self.generate_image_description(content)
                image_path = self.download_image(image_desc)
                if image_path and os.path.exists(image_path):
                    slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))

                    os.remove(image_path)
            except Exception as e:
                print(e)

        return slide

  def create_image_slide(self, title, content, image_query):
        slide_layout = self.presentation.slide_layouts[8]
        slide = self.presentation.slides.add_slide(slide_layout)

        title_box = slide.shapes.add_textbox(Inches(1), Inches(1), Inches(8), Inches(1))
        title_frame = title_box.text_frame
        title_frame.text = title
        title_frame.paragraphs[0].font.size = Pt(30)
        title_frame.paragraphs[0].font.bold = True
        title_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
        title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        content_box = slide.shapes.add_textbox(Inches(1), Inches(3), Inches(8), Inches(5))
        content_frame = content_box.text_frame
        content_frame.text = content


        for paragraph in content_frame.paragraphs:
            paragraph.font.size = Pt(20)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        try:
            image_path = self.download_image(image_query)
            if image_path and os.path.exists(image_path):
                slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))
                os.remove(image_path)
        except Exception as e:
            print(e)

        return slide

  def generate_presentation(self, topic, num_slides=5, output_file="generated_presentation.pptx"):

        outline = self.generate_content_outline(topic, num_slides)

        for i, slide_data in enumerate(outline):
            title = slide_data["title"]
            content = slide_data["content"]
            slide_type = slide_data["slide_type"]

            print(f"Creating slide {i+1}: {title}")

            if i == 0 or slide_type == "title":
                self.create_title_slide(title, "Created by Aruhi")

            elif slide_type == "image":
                img_query = self.generate_image_description(content)
                self.create_image_slide(title, content, img_query) # Corrected variable name here
            else:
                include_image = (i % 3 == 0)
                self.create_content_slide(title, content, include_image)

        self.presentation.save(output_file)
        print(f"Presentation saved as: {output_file}")

        return output_file

print(" PPTGenerator initiliased")

 PPTGenerator initiliased


In [45]:
api_key = "" #enter gemini api


In [46]:
try:
  generator = PPTGenerator(api_key = api_key)
except Exception as e:
  print(e)

In [47]:
topic = "Artificial Intelligence in Healthcare"
num_slides = 6

try:
    output_file = generator.generate_presentation(topic, num_slides, "ai_healthcare_presentation.pptx")
except Exception as e:
    print(e)


Creating slide 1: Artificial Intelligence in Healthcare: Revolutionizing Patient Care
Creating slide 2: What is AI and Why Healthcare?
Creating slide 3: AI in Action: Key Applications
Creating slide 4: The Promise of AI: Benefits & Advantages
Creating slide 5: Navigating the Hurdles: Challenges & Ethics
Creating slide 6: The Future is Intelligent: Conclusion
Presentation saved as: ai_healthcare_presentation.pptx
